# 1. Setting

In [30]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [31]:
# 필요한 라이브러리 세팅
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta

!pip install tqdm
from tqdm import tqdm

import pandas as pd
import numpy as np

In [32]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [33]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

# url='https://www.yanolja.com/hotel'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


# 2. Crawling Review

### 2-1. 특수문자 제거, 띄어쓰기 제거, 20글자 이상 리뷰 불러오기 함수 정의

In [34]:
import re

# 특수문자 제거 함수 정의
def remove_special_character(text):
  hangul = re.compile(r'[^ ㄱ-ㅣ 가-힣]') # 정규표현식에서 특수문자 제거를 위한 유니코드 적용
  hangul = hangul.sub('', text) # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
  return hangul

In [35]:
# # 띄어쓰기 제거 함수 정의
# def apply_regular_expression_2(sentence):
#   pattern = re.compile(r'\s+') # 정규표현식에서 띄어쓰기 제거를 위한 유니코드 적용
#   result = pattern.sub('', sentence) #위에 설정한 "pattern" 규칙을 "sentence"에 적용시킴
#   return result

In [36]:
# # 20글자 이상 리뷰 모음
# # over_20이라는 새로운 리스트 생성
# over_20 = []
# # for loop이용: 특수문자와 띄어쓰기 모두 제거한 arr이라는 리스트에서 글자수 20이상인 리뷰만 추출 -> "over_20"이라는 빈 리스트에 추가
# for i in range(len(arr)):
#   if len(arr[i]) >= 20:
#     over_20.append(arr[i])
# # result: over_20리스트 안에 20글자 이상인 것만 모두 저장;
# print(over_20)

### 2-2. 리뷰 크롤링 함수 정의

In [37]:
def change_date(x):
    now = datetime.datetime.today()
    today = now.strftime("%Y. %m. %d")
    yesterday = (now - timedelta(days=1)).strftime("%Y. %m. %d")
    _2days_ago = (now - timedelta(days = 2)).strftime("%Y. %m. %d")
    _3days_ago = (now - timedelta(days = 3)).strftime("%Y. %m. %d")
    hour = int(datetime.datetime.today().strftime("%H"))
    min = int(datetime.datetime.today().strftime("%M"))
    if "분 전" in x:
        x = int(x.replace("분 전", ""))
        if min - x > 0: return today
        else: return yesterday
    if "시간 전" in x:
        x = int(x.replace("시간 전", ""))
        if hour - x > 0: return today
        else: return yesterday
    elif "일 전" in x:
        if "1" in x: return yesterday
        elif "2" in x: return _2days_ago
        else: return _3days_ago
    else: return x

In [38]:
def yanolja_crawl(url):
  global review_df # 전역 변수를 지역 범위에서 적용
  driver.get(url)

  hotel = driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf h1').text
  hotel = hotel.replace("[★숙박대전] ", "")

  # 리뷰 페이지 클릭 (Click review page)
  driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf > a').click()

  # 무한 스크롤
  time.sleep(1)

  SCROLL_PAUSE_TIME = 1.5

  # Get scroll height : 스크롤 높이 가져오기
  last_height = driver.execute_script("return document.body.scrollHeight")

  while True:
      # Scroll down to bottom : 스크롤 끝까지 다운
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

      # Wait to load page : 1.5초 대기
      time.sleep(SCROLL_PAUSE_TIME)
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
      time.sleep(SCROLL_PAUSE_TIME)

      # Calculate new scroll height and compare with last scroll height : 스크롤 다운 후 스크롤 높이 다시 가져오기
      new_height = driver.execute_script("return document.body.scrollHeight")

      if new_height == last_height:
        break
      
      last_height = new_height

  # 스크롤 끝나고 페이지 elements 가져오기
  raw = driver.page_source # 페이지의 elements 모두 가져오기
  html = BeautifulSoup(raw, "html.parser")

  # 리뷰 텍스트 선택자: div.item-wrap
  reviews = html.select("div.item-wrap")

  columns = ['hotel', 'star', 'review', 'clean_review', 'date']

  for r in reviews:
      review = r.select_one("p")
      date = r.select_one('time')
      date = change_date(date.text)
      year = float(date.split('.')[0]) # date.text : 2020. 01. 01 형태
      month = float(date.split('.')[1])

      stars = r.select('.container.score')
      for s in stars:
          star_count = 5 - len(s.select(".ico.gray"))
      
      # 특수문자 제거하고, 20글자 이상인 리뷰들만 df 생성
      review20 = remove_special_character(review.text)
      
      if len(review20) >= 15 and len(review20) <= 500: # 리뷰가 15자 이상 500자 이하, 날짜가 2019.06. 이후
        review_df = review_df.append(pd.DataFrame([[hotel, star_count, review.text, review20, date]], columns = columns),
                      ignore_index=True)

# 3. 제주 호텔 리스트 불러와서 함수 적용

In [39]:
hotel_list_url = '/content/drive/MyDrive/Codepresso/TeamProject/data/Seoul/seoul_hotel_list.csv'
hotel_list = pd.read_csv(hotel_list_url)
hotel_list = hotel_list[['hotel_code', 'hotel_name']]
hotel_list['hotel_name'][17] = '소피텔 앰배서더 서울 호텔&서비스드 레지던스'
hotel_list

,hotel_code,hotel_name
0,3001395,신라스테이 역삼
1,3001615,카파스 호텔
2,3000769,노보텔 앰배서더 강남
3,3001437,호텔 더 디자이너스 리즈 강남
4,3009050,신라스테이 서초
5,1000105844,신라스테이 삼성
6,3000742,이비스 스타일 앰배서더 강남
7,3018171,호텔 인 나인 강남
8,3011846,L7 강남 바이 롯데
9,1000101062,신천(잠실새내) 호텔 더 캐슬


In [40]:
# url_list에 호텔 코드 모두 넣어서 리스트 생성
url_list = []

for i in range(0, len(hotel_list)):
  url_list.append('https://www.yanolja.com/hotel/'+ np.str(hotel_list['hotel_code'][i]))
del url_list[17]

In [41]:
len(url_list)

27

In [42]:
# yanolja_crawl 함수 불러오기 전에 review_df(DataFrame) 정의
columns = ['hotel', 'star', 'review', 'clean_review', 'date']
review_df = pd.DataFrame(columns = columns) # 데이터프레임 생성

for url in tqdm(url_list[:28]): #0~40
  yanolja_crawl(url)

review_df.to_csv('/content/drive/MyDrive/Codepresso/TeamProject/data/Seoul/seoul_hotel_review_raw.csv')

driver.close()

100%|██████████| 2/2 [01:26<00:00, 43.09s/it]


In [95]:
review_url = '/content/drive/MyDrive/Codepresso/TeamProject/data/Seoul/seoul_hotel_review_raw.csv'
hotel_review_df = pd.read_csv(review_url)

In [96]:
hotel_review_df = hotel_review_df.drop([hotel_review_df.columns[0]], axis='columns')
hotel_review_df.tail()

,hotel,star,review,clean_review,date
12745,롯데호텔 월드,4,오래된 호텔 느낌은 있었지만 침대랑 침구가 정말 편했어요 그리고 롯데월드 패키지 했...,오래된 호텔 느낌은 있었지만 침대랑 침구가 정말 편했어요 그리고 롯데월드 패키지 했...,2018. 06. 01
12746,롯데호텔 월드,5,"프론트 데스크 직원분들 너무 친절하셨어요!!\n패키지라서 조식, 아쿠아리움 포함이었...",프론트 데스크 직원분들 너무 친절하셨어요패키지라서 조식 아쿠아리움 포함이었는데 만족했어요,2018. 04. 16
12747,롯데호텔 월드,5,"잠실역이랑 이어져있고 롯데월드, 타워, 석촌호수와 완전 가깝기네 교통은 말할것도 없...",잠실역이랑 이어져있고 롯데월드 타워 석촌호수와 완전 가깝기네 교통은 말할것도 없습니...,2018. 04. 13
12748,롯데호텔 월드,3,방문아래가떠있어서 소리가다들려요,방문아래가떠있어서 소리가다들려요,2018. 04. 02
12749,롯데호텔 월드,4,"접근성, 교통, 편의성 다 만족합니다. 하지만 화장실 변기 수압이 ㅜㅜ 너무 약했어요",접근성 교통 편의성 다 만족합니다 하지만 화장실 변기 수압이 ㅜㅜ 너무 약했어요,2017. 12. 09


In [45]:
hotel_review_df.to_csv('/content/drive/MyDrive/Codepresso/TeamProject/data/Seoul/seoul_hotel_review_raw.csv')

In [103]:
seoul_review_df_bad = hotel_review_df[hotel_review_df['star'] == 1]

In [104]:
seoul_review_df_bad = seoul_review_df_bad.append(hotel_review_df[hotel_review_df['star'] == 2])

In [105]:
seoul_review_df_bad = seoul_review_df_bad.sort_index(ignore_index=True)
seoul_review_df_bad

,hotel,star,review,clean_review,date
0,신라스테이 역삼,2,프론트 스탭 분들은 친절하게 서비스 해 주시려고 해 주셔서 감사했습니다.! 신라스테...,프론트 스탭 분들은 친절하게 서비스 해 주시려고 해 주셔서 감사했습니다 신라스테이 ...,2021. 08. 11
1,신라스테이 역삼,1,서랍위에 있어서 손님용이냐고 카운터에 물어보니 아니랍니다. 저 칸도 손님이 사용하는...,서랍위에 있어서 손님용이냐고 카운터에 물어보니 아니랍니다 저 칸도 손님이 사용하는 ...,2021. 08. 10
2,신라스테이 역삼,1,벽에 보풀이낫나했는데\n먼지가 세상세상 심하게다붙어잇고..\n청소상태 진짜 초ㅣ악이...,벽에 보풀이낫나했는데먼지가 세상세상 심하게다붙어잇고청소상태 진짜 초ㅣ악이네요머리카락...,2021. 07. 25
3,신라스테이 역삼,2,첨에 체크인한 방에 쓰레기가 있어서 방 바꿔주셨어요,첨에 체크인한 방에 쓰레기가 있어서 방 바꿔주셨어요,2021. 06. 02
4,신라스테이 역삼,1,다 좋은데\n방에 게속 웅~웅 거리는 소음때문에\n잠을 한숨도 못잠.,다 좋은데방에 게속 웅웅 거리는 소음때문에잠을 한숨도 못잠,2021. 06. 02
...,...,...,...,...,...
809,호텔티롤,2,깨끗하고 가족끼리 머물기 무난함.난방이 너무더움.,깨끗하고 가족끼리 머물기 무난함난방이 너무더움,2019. 02. 10
810,호텔티롤,2,후론트에 계신분은 너무너무너무 친절하시고 저렴한 가격에 좋은곳에서 잤는데 이불에검정...,후론트에 계신분은 너무너무너무 친절하시고 저렴한 가격에 좋은곳에서 잤는데 이불에검정...,2019. 01. 13
811,호텔티롤,2,옆건물 낮은층에 묶었는데 1층은 유흥업소며 별관인 그곳은 유흥업소 입구와 함께 있어...,옆건물 낮은층에 묶었는데 층은 유흥업소며 별관인 그곳은 유흥업소 입구와 함께 있어서...,2019. 01. 06
812,롯데호텔 월드,2,한달전에 예약했음에도 불과하고 롯데월드 초록색 지붕만 보이는 이상한 뷰로 배정이 되...,한달전에 예약했음에도 불과하고 롯데월드 초록색 지붕만 보이는 이상한 뷰로 배정이 되...,2021. 04. 11


In [110]:
seoul_review_df_good = hotel_review_df[hotel_review_df['star'] == 5]

In [111]:
# 호텔별 리뷰 200개 random 추출
import random

seoul_review_df_good = seoul_review_df_good.sample(n=814, random_state=1000)

In [112]:
seoul_review_df_good = seoul_review_df_good.sort_index(ignore_index=True)
seoul_review_df_good

,hotel,star,review,clean_review,date
0,신라스테이 역삼,5,2주에 한번씩 이용하는 신라스테이 역삼~~ 모두 다 좋은데 확실히 연식이 조금 되다...,주에 한번씩 이용하는 신라스테이 역삼 모두 다 좋은데 확실히 연식이 조금 되다보니 ...,2021. 08. 17
1,신라스테이 역삼,5,호캉스는 여기 좋내요\n조식을 못먹어서 아쉽내요 늦게 일어나서 ㅠ,호캉스는 여기 좋내요조식을 못먹어서 아쉽내요 늦게 일어나서 ㅠ,2021. 08. 02
2,신라스테이 역삼,5,친절한 직원 깔끔한 시설 좋은 위치접근성 좋았어요!,친절한 직원 깔끔한 시설 좋은 위치접근성 좋았어요,2021. 08. 01
3,신라스테이 역삼,5,좋은 룸컨디션과 넉넉한 주차공간 좋았어요!,좋은 룸컨디션과 넉넉한 주차공간 좋았어요,2021. 07. 15
4,신라스테이 역삼,5,제주도 가기전에 이틀읓 신라스테이에서 묵었는데 너무 편하게 잘 머물렀습니다 ㅎㅎ\n...,제주도 가기전에 이틀읓 신라스테이에서 묵었는데 너무 편하게 잘 머물렀습니다 ㅎㅎ이날...,2021. 07. 05
...,...,...,...,...,...
809,롯데호텔 월드,5,롯데월드 가려고 예약했었는데 역시 5성급은 역시나입니다,롯데월드 가려고 예약했었는데 역시 성급은 역시나입니다,2019. 11. 17
810,롯데호텔 월드,5,롯데월드랑 함께 이용했습니다\n정말 좋았어요,롯데월드랑 함께 이용했습니다정말 좋았어요,2018. 12. 09
811,롯데호텔 월드,5,조아요 시설도 좋구 깔끔해요,조아요 시설도 좋구 깔끔해요,2018. 12. 09
812,롯데호텔 월드,5,"패키지 상품으로 재미도 즐기고, 호텔도 편히 쉬었어요.",패키지 상품으로 재미도 즐기고 호텔도 편히 쉬었어요,2018. 11. 28


# 전처리

In [68]:
import re

# 자음/모음 제거 함수 정의
def clean_text(text):
  hangul = re.sub('[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅃㅉㄸㄲㅆㅛㅕㅑㅐㅔㅗㅓㅏㅣㅜㅠㅡ]', '', text) # 정규표현식에서 자음, 모음 제거
  return hangul

In [69]:
# import re

# 이모티콘 제거 함수 정의
def remove_emoticon(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    hangul = emoji_pattern.sub(r'', text)
    return hangul

In [115]:
# 새로운 리스트 생성
clean_review_good = []
clean_review_bad = []

# 특수문자 제거 완료된 리뷰 불러오기
clean_review_col_good = seoul_review_df_good['clean_review'].copy()
clean_review_col_bad = seoul_review_df_bad['clean_review'].copy()

# for loop으로 위의 두 함수를 적용한 객체들만 clean_review 리스트에 추가
for i in range(len(clean_review_col_good)):
  temp_1 = clean_text(clean_review_col_good[i])
  temp_2 = remove_emoticon(temp_1)
  clean_review_good.append(temp_2)

for i in range(len(clean_review_col_bad)):
  temp_1 = clean_text(clean_review_col_bad[i])
  temp_2 = remove_emoticon(temp_1)
  clean_review_bad.append(temp_2)

In [116]:
# import libraries
!pip install git+https://github.com/ssut/py-hanspell.git
from tqdm import tqdm
from hanspell import spell_checker
from hanspell.constants import CheckResult

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-sn63ev6z
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-sn63ev6z
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=61924b8eeaf1591d6d5b3b36346cce5089a2f0a1e9bfa691b61bcf2f07e76e7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-51e7hnl9/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [118]:
# 맞춤법 검사 함수 정의
def spell_check(review):
    result = spell_checker.check(review)
    return result.checked

In [124]:
# 기존의 clean_review 컬럼을 전처리(자음/모음 제거, 이모티콘 제거)된 것으로 교체
seoul_review_df_good = seoul_review_df_good.assign(clean_review_good = [r for r in clean_review_good])
seoul_review_df_bad = seoul_review_df_bad.assign(clean_review_bad = [r for r in clean_review_bad])

In [125]:
# 맞춤법 검사 함수 적용한 preprocessed_review 컬럼 생성
seoul_review_df_good['preprocessed_review'] = seoul_review_df_good['clean_review_good'].apply(spell_check)

seoul_review_df_good = seoul_review_df_good[['hotel', 'star', 'review', 'preprocessed_review', 'date']] # 필요한 컬럼만 추출
seoul_review_df_good

,hotel,star,review,preprocessed_review,date
0,신라스테이 역삼,5,2주에 한번씩 이용하는 신라스테이 역삼~~ 모두 다 좋은데 확실히 연식이 조금 되다...,주에 한 번씩 이용하는 신라스테이 역삼 모두 다 좋은데 확실히 연식이 조금 되다 보...,2021. 08. 17
1,신라스테이 역삼,5,호캉스는 여기 좋내요\n조식을 못먹어서 아쉽내요 늦게 일어나서 ㅠ,호캉스는 여기 좋네요 조식을 못 먹어서 아쉽네요 늦게 일어나서,2021. 08. 02
2,신라스테이 역삼,5,친절한 직원 깔끔한 시설 좋은 위치접근성 좋았어요!,친절한 직원 깔끔한 시설 좋은 위치 접근성 좋았어요,2021. 08. 01
3,신라스테이 역삼,5,좋은 룸컨디션과 넉넉한 주차공간 좋았어요!,좋은 룸 컨디션과 넉넉한 주차공간 좋았어요,2021. 07. 15
4,신라스테이 역삼,5,제주도 가기전에 이틀읓 신라스테이에서 묵었는데 너무 편하게 잘 머물렀습니다 ㅎㅎ\n...,제주도 가기 전에 이틀읓 신라스테이에서 묵었는데 너무 편하게 잘 머물렀습니다 이날 ...,2021. 07. 05
...,...,...,...,...,...
809,롯데호텔 월드,5,롯데월드 가려고 예약했었는데 역시 5성급은 역시나입니다,롯데월드 가려고 예약했었는데 역시 성급은 역시나입니다,2019. 11. 17
810,롯데호텔 월드,5,롯데월드랑 함께 이용했습니다\n정말 좋았어요,롯데월드랑 함께 이용했습니다 정말 좋았어요,2018. 12. 09
811,롯데호텔 월드,5,조아요 시설도 좋구 깔끔해요,좋아요 시설도 좋고 깔끔해요,2018. 12. 09
812,롯데호텔 월드,5,"패키지 상품으로 재미도 즐기고, 호텔도 편히 쉬었어요.",패키지 상품으로 재미도 즐기고 호텔도 편히 쉬었어요,2018. 11. 28


In [126]:
# 맞춤법 검사 함수 적용한 preprocessed_review 컬럼 생성
seoul_review_df_bad['preprocessed_review'] = seoul_review_df_bad['clean_review_bad'].apply(spell_check)

seoul_review_df_bad = seoul_review_df_bad[['hotel', 'star', 'review', 'preprocessed_review', 'date']] # 필요한 컬럼만 추출
seoul_review_df_bad

,hotel,star,review,preprocessed_review,date
0,신라스테이 역삼,2,프론트 스탭 분들은 친절하게 서비스 해 주시려고 해 주셔서 감사했습니다.! 신라스테...,프런트 스태프 분들은 친절하게 서비스해 주시려고 해 주셔서 감사했습니다 신라스테이 ...,2021. 08. 11
1,신라스테이 역삼,1,서랍위에 있어서 손님용이냐고 카운터에 물어보니 아니랍니다. 저 칸도 손님이 사용하는...,서랍 위에 있어서 손님 용이냐고 카운터에 물어보니 아니랍니다 저 칸도 손님이 사용하...,2021. 08. 10
2,신라스테이 역삼,1,벽에 보풀이낫나했는데\n먼지가 세상세상 심하게다붙어잇고..\n청소상태 진짜 초ㅣ악이...,벽에 보풀이 낫나 했는데 먼지가 세상 세상 심하게 다붙어 잇고 청소상태 진짜 최악이...,2021. 07. 25
3,신라스테이 역삼,2,첨에 체크인한 방에 쓰레기가 있어서 방 바꿔주셨어요,첨에 체크인 한 방에 쓰레기가 있어서 방 바꿔주셨어요,2021. 06. 02
4,신라스테이 역삼,1,다 좋은데\n방에 게속 웅~웅 거리는 소음때문에\n잠을 한숨도 못잠.,다 좋은데 방에 계속 윙윙 거리는 소음 때문에 잠을 한숨도 못 잠,2021. 06. 02
...,...,...,...,...,...
809,호텔티롤,2,깨끗하고 가족끼리 머물기 무난함.난방이 너무더움.,깨끗하고 가족끼리 머물기 무난함 난방이 너무 더움,2019. 02. 10
810,호텔티롤,2,후론트에 계신분은 너무너무너무 친절하시고 저렴한 가격에 좋은곳에서 잤는데 이불에검정...,프런트에 계신 분은 너무너무너무 친절하시고 저렴한 가격에 좋은 곳에서 잤는데 이불에...,2019. 01. 13
811,호텔티롤,2,옆건물 낮은층에 묶었는데 1층은 유흥업소며 별관인 그곳은 유흥업소 입구와 함께 있어...,옆 건물 낮은 층에 묶었는데 층은 유흥업소며 별관인 그곳은 유흥업소 입구와 함께 있...,2019. 01. 06
812,롯데호텔 월드,2,한달전에 예약했음에도 불과하고 롯데월드 초록색 지붕만 보이는 이상한 뷰로 배정이 되...,한 달 전에 예약했음에도 불과하고 롯데월드 초록색 지붕만 보이는 이상한 뷰로 배정이...,2021. 04. 11


In [127]:
# 가성비 -> 구성비 맞춤법 검사기 오류 + 뷰 -> 경치 : 형태소 분석을 위한 처리
cost = ['구성비', '가성비', '가성 비', '가 성비', '가성', '가격 대비']

for i in range(len(seoul_review_df_good)):
  for j in range(len(cost)):
    if cost[j] in seoul_review_df_good['preprocessed_review'][i]:
      seoul_review_df_good['preprocessed_review'][i] = seoul_review_df_good['preprocessed_review'][i].replace(cost[j], " 구성비 ")
  if '뷰' in seoul_review_df_good['preprocessed_review'][i]:
      seoul_review_df_good['preprocessed_review'][i] = seoul_review_df_good['preprocessed_review'][i].replace("뷰", "경치")
  if '더 보기' in seoul_review_df_good['preprocessed_review'][i]:
      seoul_review_df_good['preprocessed_review'][i] = seoul_review_df_good['preprocessed_review'][i].replace("더 보기", "")

for i in range(len(seoul_review_df_bad)):
  for j in range(len(cost)):
    if cost[j] in seoul_review_df_bad['preprocessed_review'][i]:
      seoul_review_df_bad['preprocessed_review'][i] = seoul_review_df_bad['preprocessed_review'][i].replace(cost[j], " 구성비 ")
  if '뷰' in seoul_review_df_bad['preprocessed_review'][i]:
      seoul_review_df_bad['preprocessed_review'][i] = seoul_review_df_bad['preprocessed_review'][i].replace("뷰", "경치")
  if '더 보기' in seoul_review_df_bad['preprocessed_review'][i]:
      seoul_review_df_bad['preprocessed_review'][i] = seoul_review_df_bad['preprocessed_review'][i].replace("더 보기", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib

In [128]:
seoul_review_df_good.to_csv('/content/drive/MyDrive/Codepresso/TeamProject/data/Seoul/seoul_hotel_review_good_preprocessed.csv')
seoul_review_df_bad.to_csv('/content/drive/MyDrive/Codepresso/TeamProject/data/Seoul/seoul_hotel_review_bad_preprocessed.csv')